# Model Usage

Use the mode through the API.

### Libraries

In [21]:
import pandas as pd
import json
import requests
import numpy as np

### Read test data

In [45]:
model_variables =['nombre_paquete',
                  'ciudad_agencia',
                  'nombre_tipo_habitacion',
                  'nombre_canal',
                  'nombre_estado'
                  ]


# Read sample data choosen randomly from the dataset
test_model_data = pd.read_parquet("reservaciones_eda.parquet").sample(n=1)

# Convert the dataframe to a dictionary
test_model_dict = test_model_data.to_dict(orient='list')

### 1. Convert all dataframe into string format
for key in test_model_dict:
    test_model_dict[key] = [str(value) for value in test_model_dict[key]]


### 2. Convert to json format converting all records to strings format
data_json = json.dumps({"data": [test_model_dict]})


### 2. Load the scoring URI from a file
suri = open("uri.json", "r")
scoring_uri = json.load(suri)["URI"][0]
suri.close()

### 3. Use the requests library to send a POST request to the model endpoint
headers = {"Content-Type": "application/json"}

response = requests.post(scoring_uri, data=data_json, headers=headers)


if response.status_code == 200:
    result = json.loads(response.json())
    print(result)
    result = "Reservación Cancelada" if result['result'][0]==1 else "Reservación No Cancelada"
    
    test_model_data["Cancelacion prediction"] = result
    
    #display(test_model_data[["id_reservaciones", "nombre_estatus_reservacion", "Cancelacion prediction"]])
    display(test_model_data[model_variables + ["Cancelacion prediction"]])
else:
    print(f"Error: {response.text}")

{'result': [1]}


,nombre_paquete,ciudad_agencia,nombre_tipo_habitacion,nombre_canal,nombre_estado,Cancelacion prediction
46704,Walk In,Leon,Estd Db Sn12ast,Lada 800 Nacional68,Guerrero,Reservación Cancelada


In [46]:
test_model_data.nombre_paquete.value_counts()

nombre_paquete
Walk In          1
Entre Semana     0
Fin De Semana    0
Lunamielero      0
Ninguno          0
Sin Definir      0
Name: count, dtype: int64